In [ ]:
# --- Dependencias principales ---
import numpy  # Operaciones numéricas
import surprise  # Algoritmos de recomendación
import pandas as pd  # Manipulación de datos
from surprise import Reader, Dataset, SVD  # Componentes de Surprise
from surprise.model_selection import train_test_split  # División de datos
from surprise import accuracy  # Métricas
import os  # Operaciones de sistema
import mlflow  # Seguimiento de experimentos
import mlflow.sklearn  # Registro de modelos sklearn
from pathlib import Path  # Manejo robusto de rutas

# --- Información de entorno ---
print(f"Numpy version: {numpy.__version__}")
print(f"Surprise version: {surprise.__version__}")
print("Librerías listas, incluyendo MLflow, SVD y Pathlib.")

# --- Resolución de la raíz del proyecto ---
# Permite ejecutar el notebook desde distintos contextos sin perder referencias
try:
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
except NameError:
    if "notebooks" in os.getcwd():
        BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))
    else:
        BASE_DIR = os.getcwd()
print(f"Raíz del proyecto detectada en: {BASE_DIR}")

# --- Configuración universal de MLflow ---
# Se traduce la ruta local a URI universal para compatibilidad multiplataforma
mlruns_path = Path(BASE_DIR) / 'mlruns'
tracking_uri = mlruns_path.as_uri()
mlflow.set_tracking_uri(tracking_uri)
print(f"MLflow configurado para guardar en la dirección universal: {tracking_uri}")
mlflow.set_experiment("LatentLens-SVD-Evaluation")

# --- Carga y muestreo de datos MovieLens ---
# Selecciona los usuarios y películas más activos para reducir la dispersión y el tamaño
ratings_path = Path(BASE_DIR) / 'data' / 'ml-25m' / 'ratings.csv'
ratings_df = pd.read_csv(ratings_path)

n_users = 40000  # Número de usuarios a considerar
n_movies = 20000  # Número de películas a considerar
user_ids = ratings_df['userId'].value_counts().nlargest(n_users).index
movie_ids = ratings_df['movieId'].value_counts().nlargest(n_movies).index
sampled_df = ratings_df[(ratings_df['userId'].isin(user_ids)) & (ratings_df['movieId'].isin(movie_ids))]

# --- Conversión a formato Surprise ---
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(sampled_df[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# --- Definición de experimento y entrenamiento SVD ---
run_name = "SVD_n_factors_150_epochs_20"
with mlflow.start_run(run_name=run_name) as run:
    print(f"Iniciando nuevo run de MLflow: '{run_name}' (ID: {run.info.run_id})")
    # Hiperparámetros principales
    n_factors = 150  # Dimensiones latentes
    n_epochs = 20    # Iteraciones de entrenamiento
    mlflow.log_param("model_type", "SVD")
    mlflow.log_param("n_factors", n_factors)
    mlflow.log_param("n_epochs", n_epochs)
    # Instanciación y entrenamiento del modelo
    model = SVD(n_factors=n_factors, n_epochs=n_epochs, random_state=42)
    print(f"Entrenando SVD con {n_factors} factores...")
    model.fit(trainset)
    print("¡Modelo entrenado!")
    # Evaluación cuantitativa
    predictions = model.test(testset)
    rmse = accuracy.rmse(predictions)
    mlflow.log_metric("rmse", rmse)
    # Registro del modelo entrenado como artefacto
    mlflow.sklearn.log_model(model, "surprise_svd_model")
    print("Modelo SVD (artefacto) y métricas registradas en MLflow.")

# --- Fin de ejecución ---
print(f"\n¡Experimento '{run_name}' finalizado!")


Numpy version: 1.26.4
Surprise version: 1.1.4
Librerías listas, incluyendo MLflow, SVD y Pathlib.
Raíz del proyecto detectada en: c:\Users\Gat\Documents\GitHub\LatentLens
MLflow configurado para guardar en la dirección universal: file:///c:/Users/Gat/Documents/GitHub/LatentLens/mlruns
Iniciando nuevo run de MLflow: 'SVD_n_factors_150_epochs_20' (ID: 5e2fe0b315ae405d922b102417191110)
Entrenando SVD con 150 factores...
¡Modelo entrenado!


2025/08/09 14:53:06 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


RMSE: 0.7433


2025/08/09 14:53:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo SVD (artefacto) y métricas registradas en MLflow.

¡Experimento 'SVD_n_factors_150_epochs_20' finalizado!
